# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint

# Import API key
from api_keys import g_key
# Configure gmaps
gmaps.configure(api_key=g_key)


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Store filepath in a variable

Import_CityDatafile = "OutputData/CityData.csv"
# Read our Data file with the pandas library

City_data_df = pd.read_csv(Import_CityDatafile)
City_data_df.head() 


,City,latitude,Longitude,Maxtemp,Humidity,Cloudiness,Windspeed,Country,Date
0,Saint-Pierre,-21.34,55.48,19.00,88,75,9.30,RE,1595103628
1,Itaituba,-4.28,-55.98,35.39,30,52,1.77,BR,1595103398
2,Jamestown,42.10,-79.24,30.56,58,1,4.60,US,1595103523
3,Bluff,-46.60,168.33,2.55,82,0,3.89,NZ,1595103628
4,Punta Arenas,-53.15,-70.92,2.00,80,40,8.20,CL,1595103368


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Store latitude and longitude in locations
locations = City_data_df[["latitude","Longitude"]]


Humidity=City_data_df["Humidity"]

In [4]:
#plot heatmap
fig= gmaps.figure(center= (0,0), zoom_level=3)

#create heat layer
heat_layer = gmaps.heatmap_layer(locations,weights=Humidity,
                                 dissipating=False,
                                 max_intensity=100,
                                 point_radius=1)
#Add layer
fig.add_layer(heat_layer)

#Display figure
fig


Figure(layout=FigureLayout(height='420px'))

In [5]:

# Plost as a Hybrid map
fig = gmaps.figure(map_type="HYBRID")

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations,weights=Humidity,
                                 dissipating=False,
                                 max_intensity=10,
                                 point_radius=1)

fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
#get the ideal temperature using below criteria
ideal_weather = City_data_df.loc[(City_data_df["Maxtemp"].between(20,25,inclusive=True))&
                                 (City_data_df["Windspeed"]<10)&
                                 (City_data_df["Cloudiness"]==0)
                                ]
ideal_weather.head(15)

,City,latitude,Longitude,Maxtemp,Humidity,Cloudiness,Windspeed,Country,Date
76,Arraial do Cabo,-22.97,-42.03,22.00,78,0,8.70,BR,1595103631
89,Yumen,40.28,97.20,21.11,23,0,3.42,CN,1595103633
94,Lüderitz,-26.65,15.16,21.00,21,0,3.60,NaN,1595103634
140,Soyo,-6.13,12.37,22.25,86,0,4.40,AO,1595103639
148,Hastings,50.86,0.57,20.56,82,0,1.79,GB,1595103640
168,Walvis Bay,-22.96,14.51,22.00,13,0,7.20,NaN,1595103641
199,Frederico Westphalen,-27.36,-53.39,24.44,77,0,0.89,BR,1595103645
212,Akhtubinsk,48.28,46.17,24.06,50,0,6.01,RU,1595103646
219,Marsá Maţrūḩ,31.35,27.25,25.00,78,0,2.60,EG,1595103423
297,Kasongo-Lunda,-6.47,16.82,22.31,33,0,1.06,CD,1595103652


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
#Store into variable named hotel_df.

hotel_df=pd.DataFrame(ideal_weather)
hotel_df

,City,latitude,Longitude,Maxtemp,Humidity,Cloudiness,Windspeed,Country,Date
76,Arraial do Cabo,-22.97,-42.03,22.00,78,0,8.70,BR,1595103631
89,Yumen,40.28,97.20,21.11,23,0,3.42,CN,1595103633
94,Lüderitz,-26.65,15.16,21.00,21,0,3.60,NaN,1595103634
140,Soyo,-6.13,12.37,22.25,86,0,4.40,AO,1595103639
148,Hastings,50.86,0.57,20.56,82,0,1.79,GB,1595103640
168,Walvis Bay,-22.96,14.51,22.00,13,0,7.20,NaN,1595103641
199,Frederico Westphalen,-27.36,-53.39,24.44,77,0,0.89,BR,1595103645
212,Akhtubinsk,48.28,46.17,24.06,50,0,6.01,RU,1595103646
219,Marsá Maţrūḩ,31.35,27.25,25.00,78,0,2.60,EG,1595103423
297,Kasongo-Lunda,-6.47,16.82,22.31,33,0,1.06,CD,1595103652


In [8]:
# Add columns for "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_data=hotel_df.head(10)
hotel_data

,City,latitude,Longitude,Maxtemp,Humidity,Cloudiness,Windspeed,Country,Date,Hotel Name
76,Arraial do Cabo,-22.97,-42.03,22.00,78,0,8.70,BR,1595103631,
89,Yumen,40.28,97.20,21.11,23,0,3.42,CN,1595103633,
94,Lüderitz,-26.65,15.16,21.00,21,0,3.60,NaN,1595103634,
140,Soyo,-6.13,12.37,22.25,86,0,4.40,AO,1595103639,
148,Hastings,50.86,0.57,20.56,82,0,1.79,GB,1595103640,
168,Walvis Bay,-22.96,14.51,22.00,13,0,7.20,NaN,1595103641,
199,Frederico Westphalen,-27.36,-53.39,24.44,77,0,0.89,BR,1595103645,
212,Akhtubinsk,48.28,46.17,24.06,50,0,6.01,RU,1595103646,
219,Marsá Maţrūḩ,31.35,27.25,25.00,78,0,2.60,EG,1595103423,
297,Kasongo-Lunda,-6.47,16.82,22.31,33,0,1.06,CD,1595103652,


In [9]:
from pprint import pprint

In [10]:
# params dictionary to update each iteration
params = {
    
    "radius": 5000,
    #"keyword": "hotels",
    "types":"lodging",
    "key": g_key
}
# Use the lat/lng we recovered to identify hotels
for index, row in hotel_data.iterrows():
    # get lat, lng from df
    lat = row["latitude"]
    lng = row["Longitude"]
 # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"    
     # Use the search term: "Hotel" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # make request and print url
    response = requests.get(base_url, params=params)
   
     # convert to json
    hotel_response= response.json()
    try:
             hotel_data.loc[index, "Hotel Name"] =  hotel_response["results"][0]["name"]
       
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
    
       
hotel_data


C:\Users\Viswajani\anaconda3\lib\site-packages\pandas\core\indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


Missing field/result... skipping.
Missing field/result... skipping.


,City,latitude,Longitude,Maxtemp,Humidity,Cloudiness,Windspeed,Country,Date,Hotel Name
76,Arraial do Cabo,-22.97,-42.03,22.00,78,0,8.70,BR,1595103631,Pousada Porto Praia
89,Yumen,40.28,97.20,21.11,23,0,3.42,CN,1595103633,
94,Lüderitz,-26.65,15.16,21.00,21,0,3.60,NaN,1595103634,LÜDERITZ NEST HOTEL
140,Soyo,-6.13,12.37,22.25,86,0,4.40,AO,1595103639,Hotel Mpampa
148,Hastings,50.86,0.57,20.56,82,0,1.79,GB,1595103640,Zanzibar Hotel
168,Walvis Bay,-22.96,14.51,22.00,13,0,7.20,NaN,1595103641,Oyster Box Guesthouse
199,Frederico Westphalen,-27.36,-53.39,24.44,77,0,0.89,BR,1595103645,Hotel Palace Frederico
212,Akhtubinsk,48.28,46.17,24.06,50,0,6.01,RU,1595103646,Vzlet
219,Marsá Maţrūḩ,31.35,27.25,25.00,78,0,2.60,EG,1595103423,Beau Site
297,Kasongo-Lunda,-6.47,16.82,22.31,33,0,1.06,CD,1595103652,


In [11]:
# Save Data to csv
hotel_data.to_csv("Hotels_info.csv")


In [12]:
#Plot the hotels on top of the humidity heatmap with each pin containing the **Hotel Name**, **City**, and **Country**.

In [13]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
#hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
#locations = hotel_df[["Lat", "Lng"]]
hotel_info = [info_box_template.format(**row) for index, row in hotel_data.iterrows()]
locations = hotel_data[["latitude", "Longitude"]]
hotel_info

['\n<dl>\n<dt>Name</dt><dd>Pousada Porto Praia</dd>\n<dt>City</dt><dd>Arraial do Cabo</dd>\n<dt>Country</dt><dd>BR</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd></dd>\n<dt>City</dt><dd>Yumen</dd>\n<dt>Country</dt><dd>CN</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>LÜDERITZ NEST HOTEL</dd>\n<dt>City</dt><dd>Lüderitz</dd>\n<dt>Country</dt><dd>nan</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>Hotel Mpampa</dd>\n<dt>City</dt><dd>Soyo</dd>\n<dt>Country</dt><dd>AO</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>Zanzibar Hotel</dd>\n<dt>City</dt><dd>Hastings</dd>\n<dt>Country</dt><dd>GB</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>Oyster Box Guesthouse</dd>\n<dt>City</dt><dd>Walvis Bay</dd>\n<dt>Country</dt><dd>nan</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>Hotel Palace Frederico</dd>\n<dt>City</dt><dd>Frederico Westphalen</dd>\n<dt>Country</dt><dd>BR</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>Vzlet</dd>\n<dt>City</dt><dd>Akhtubinsk</dd>\n<dt>Country</dt><dd>RU</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>Bea

In [14]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
#fig = gmaps.figure()
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(height='420px'))